# The BASELINE model

## Imports

In [1]:
import os
import math
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from typing import List
from torch import Tensor
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

## Config

In [2]:
from dataclasses import dataclass, field

@dataclass
class Config:
    PLAYERS_SIZE: int = 0
    CONTEXT_LEN: int = 16  ## predict the 21st match by the 20
    PERFORMANCE_INPUT_DIM = 23
    PERFORMANCE_EMBD_DIM: int = 128
    PLAYER_INPUT_DIM: int = 25   ## univpalyer
    MATCH_INPUT_EMBD: int = 14   ## match_info
    NUM_EPOCHS: int = 100
    LEARNING_RATE: float = 1e-3
    BATCH_SIZE: int = 128
    DEVICE: str = field(default_factory=lambda: "cuda" if torch.cuda.is_available() else "cpu")
    # MODEL_SAVE_PATH: str = "/kaggle/working"
    # BASE_DIR: str = '/kaggle/input/flickr8k/'
    # CROSSATT_NUM_HEADS: int = 8
    CLS_INIT_STD: float = 0.02    ## <CLS> token initialized with std 0.02 from the mean=0
    TEST_DATASET_SIZE: int = 180
    IDLE_DEVICE: str = 'cpu'
    ACCUMULATION_STEPS = 4


In [3]:
config = Config()

## Defination of **Losses**

## Data

#### Data Processing (Normalization)

In [52]:
## INPUT
# Universal Player embedding
# Match Situation Embedding
# Form embedding --> attention to be performed on

In [53]:
venues_df = pd.read_csv("venues.csv")
venues_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'venues.csv'

In [ ]:
import pandas as pd
from sklearn import preprocessing

def handle_missing_data(df, missing_handling):
    for col, method in missing_handling.items():
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in the DataFrame.")
        if method not in ['mean', 'median', 'mode', 'drop']:
            raise ValueError("Invalid method for handling missing data")
        if method in ['mean', 'median'] and not pd.is_numeric_dtype(df[col]):
            raise ValueError(f"Column '{col}' is not numerical and cannot be imputed with {method}.")
        if method == 'mean':
            df[col].fillna(df[col].mean(), inplace=True)
        elif method == 'median':
            df[col].fillna(df[col].median(), inplace=True)
        elif method == 'mode':
            df[col].fillna(df[col].mode()[0], inplace=True)
        elif method == 'drop':
            df.dropna(subset=[col], inplace=True)
    return df

def clean_data(df, operations):
    for op in operations:
        if op['type'] == 'remove_duplicates':
            df.drop_duplicates(inplace=True)
        elif op['type'] == 'convert_type':
            col = op['column']
            new_type = op['new_type']
            df[col] = df[col].astype(new_type)
        elif op['type'] == 'drop_columns':
            cols = op['columns']
            df.drop(columns=cols, inplace=True)
        elif op['type'] == 'rename_columns':
            mapping = op['mapping']
            df.rename(columns=mapping, inplace=True)
        else:
            raise ValueError(f"Unknown operation type: {op['type']}")
    return df

def scale_numerical_columns(df, numerical_cols, method='min_max'):
    if method == 'min_max':
        scaler = preprocessing.MinMaxScaler()
    elif method == 'standard':
        scaler = preprocessing.StandardScaler()
    else:
        raise ValueError("Invalid scaling method")
    for col in numerical_cols:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in the DataFrame.")
        if not pd.is_numeric_dtype(df[col]):
            raise ValueError(f"Column '{col}' is not numerical and cannot be scaled.")
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
    return df

def encode_categorical_columns(df, categorical_cols, method='one_hot'):
    if method == 'one_hot':
        df = pd.get_dummies(df, columns=categorical_cols)
    elif method == 'label':
        for col in categorical_cols:
            if col not in df.columns:
                raise ValueError(f"Column '{col}' not found in the DataFrame.")
            if df[col].dtype == 'object':
                df[col] = pd.factorize(df[col])[0]
            else:
                raise ValueError(f"Column '{col}' is not categorical and cannot be label encoded.")
    else:
        raise ValueError("Invalid encoding method")
    return df

def preprocess_csv(input_file, output_file, missing_handling=None, cleaning_operations=None, numerical_cols=None, categorical_cols=None, scale_numerical=False, encode_categorical=False):
    df = pd.read_csv(input_file)
    if missing_handling:
        df = handle_missing_data(df, missing_handling)
    if cleaning_operations:
        df = clean_data(df, cleaning_operations)
    if scale_numerical:
        if not numerical_cols:
            raise ValueError("numerical_cols must be provided if scale_numerical is True")
        df = scale_numerical_columns(df, numerical_cols)
    if encode_categorical:
        if not categorical_cols:
            raise ValueError("categorical_cols must be provided if encode_categorical is True")
        df = encode_categorical_columns(df, categorical_cols)
    df.to_csv(output_file, index=False)

# # Example usage
# input_file = 'data.csv'
# output_file = 'preprocessed_data.csv'
# missing_handling = {'age': 'mean', 'income': 'median'}
# cleaning_operations = [{'type': 'remove_duplicates'}, {'type': 'convert_type', 'column': 'age', 'new_type': 'int'}, {'type': 'drop_columns', 'columns': ['id']}]
# numerical_cols = ['age', 'income']
# categorical_cols = ['gender', 'country']
# preprocess_csv(input_file, output_file, missing_handling, cleaning_operations, numerical_cols, categorical_cols, scale_numerical=True, encode_categorical=True)

In [ ]:
dfp = pd.read_csv("players.csv")
dfp.head()

In [ ]:
dfm = pd.read_csv("matches.csv")
dfm.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler()

In [ ]:
scaling.fit_transform(dfp[["all col except the occpuation one hot"]])

In [ ]:
scaling.fit_transform(dfm['all col'])

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('updated_players.csv')

# Display the DataFrame
print("Original DataFrame:")
print(df.head())  # Show the first few rows


In [ ]:

df = df.drop(columns=['batting_style', 'bowling_style', 'name'])

# Display the modified DataFrame
print("\nModified DataFrame:")
print(df.head())


In [ ]:
# Get the list of all column names
columns = df.columns.tolist()

# Print the column names
print(columns)
len(columns)

In [ ]:
# Define lists of positions for the various groups (using lowercase for comparison)
allrounder_positions = ['allrounder', 'batting allrounder', 'bowling allrounder']
batter_positions = ['batter', 'wicketkeeper batter', 'middle-order batter', 'opening batter', 'top-order batter']
bowler_positions = ['bowler']

# Define the values to skip: do not override if player_position is "Wicketkeeper" or blank
skip_positions = ['wicketkeeper', '']

# Create a lowercase version of the player_position column (and strip any extra spaces)
df['player_position_clean'] = df['player_position'].str.lower().str.strip()

# Process all-rounders:
mask_allrounder = ~df['player_position_clean'].isin(skip_positions) & df['player_position_clean'].isin(allrounder_positions)
df.loc[mask_allrounder, ['all_rounder', 'batting', 'bowling']] = [True, False, False]

# Process batters:
mask_batter = ~df['player_position_clean'].isin(skip_positions) & df['player_position_clean'].isin(batter_positions)
df.loc[mask_batter, ['batting', 'bowling', 'all_rounder']] = [True, False, False]

# Process bowlers:
mask_bowler = ~df['player_position_clean'].isin(skip_positions) & (df['player_position_clean'] == 'bowler')
df.loc[mask_bowler, ['bowling', 'batting', 'all_rounder']] = [True, False, False]

# (Optional) Drop the helper column if you don't need it anymore
df.drop(columns=['player_position_clean'], inplace=True)

# Display the modified DataFrame
print(df.head())


In [ ]:
df.drop(columns=['player_position'], inplace=True)

# Display the modified DataFrame
print(df.head())


In [ ]:
null_counts = df.isnull().sum()

# Print the results
print("Null values in each column:")
print(null_counts)

In [ ]:
if 'strike rate in all matches combined' in df.columns:
    df['strike rate in all matches combined'] = df['strike rate in all matches combined'].fillna(0)

if 'age' in df.columns:
    median_age = df['age'].median()
    df['age'] = df['age'].fillna(median_age)

# For boolean columns: Fill missing values with False
for col in ['batting', 'bowling', 'all_rounder']:
    if col in df.columns:
        df[col] = df[col].fillna(False).infer_objects(copy=False)

if 'cricinfo_id' in df.columns:
    df['cricinfo_id'] = df['cricinfo_id'].fillna(-1)


In [ ]:
df.head()

In [ ]:
# ----- Step 3: Data Type Conversion -----
# Define numeric columns (adjust as needed)
numeric_cols = [ 'matches', 'strike rate in all matches combined', 'Total Runs Scored in all matches combined',
                'no. of fours', 'no. of sixes', 'highest runs scored in a match', 'Balls Played till now',
                'no. of dot_balls', 'maiden_overs', 'run conceded', 'Average Economy Rate', 'no. of wickets',
                'no. of balls thrown', 'highest wickets taken in a match', 'total centuries',
                'total halfcenturies', 'total catches', 'total stumps', 'total direct_runouts',
                'total indirect_runouts', 'total avg fp', 'age']

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
len(numeric_cols)


In [ ]:
# Ensure boolean columns are of type bool
bool_cols = ['batting', 'bowling', 'all_rounder']
df[bool_cols] = df[bool_cols].astype(bool)

In [ ]:
# ----- Step 4: Handle Duplicates -----
print("\nNumber of duplicate rows:", df.duplicated().sum())
df = df.drop_duplicates()


In [ ]:
# Distribution of 'age'
plt.figure(figsize=(8, 4))
sns.histplot(df['age'], kde=True, bins=20)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


In [ ]:
if 'age' in df.columns:
    median_age = df['age'].median()
    # Replace 0 values with median
    df['age'] = df['age'].replace(0, median_age)



In [ ]:
# Boxplot for 'Total Runs Scored in all matches combined'
plt.figure(figsize=(8, 4))
sns.boxplot(x=df['Total Runs Scored in all matches combined'])
plt.title('Boxplot for Total Runs Scored')
plt.xlabel('Total Runs Scored')
plt.show()

In [ ]:
# Correlation heatmap for numeric columns
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# # ----- Step 7: Data Normalization/Scaling -----
# # Scale the numeric columns using StandardScaler
# scaler = StandardScaler()
# df_scaled = df.copy()
# df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])
# print("\nSample of scaled numeric features:")
# print(df_scaled[numeric_cols].head())

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

print("\nSample of normalized numeric features:")
print(df_scaled[numeric_cols].head())


In [ ]:
# Distribution of 'age'
plt.figure(figsize=(8, 4))
sns.histplot(df_scaled['age'], kde=True, bins=20)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# Boxplot for 'Total Runs Scored in all matches combined'
plt.figure(figsize=(8, 4))
sns.boxplot(x=df_scaled['Total Runs Scored in all matches combined'])
plt.title('Boxplot for Total Runs Scored')
plt.xlabel('Total Runs Scored')
plt.show()

In [ ]:
# Correlation heatmap for numeric columns
plt.figure(figsize=(12, 10))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
selected_features = ['player_id', 'cricinfo_id'] + numeric_cols + ['batting', 'bowling', 'all_rounder']
df_selected = df_scaled[selected_features]
print("\nSelected features for further analysis/modeling:")
print(df_selected.head())

In [ ]:
df_selected.to_csv('cleaned_universal_player.csv', index=False)
print("\nCleaned data saved to 'cleaned_data.csv'")

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/content/4a8a2e3b.csv')

# Display the DataFrame
print("Original DataFrame:")
print(df.head())  # Show the first few rows


In [ ]:
# Get the list of all column names
columns = df.columns.tolist()

# Print the column names
print(columns)
len(columns)

In [ ]:
df = df.drop(columns=['date', 'event', 'teamname', 'overs_bowled'])

# Display the modified DataFrame
print("\nModified DataFrame:")
print(df.head())

In [ ]:
null_counts = df.isnull().sum()

# Print the results
print("Null values in each column:")
print(null_counts)

In [ ]:
# if 'overs_bowled' in df.columns:
#     df['overs_bowled'] = df['overs_bowled'].fillna(0)

In [ ]:
df.head()

In [ ]:
# ----- Step 3: Data Type Conversion -----
# Define numeric columns (adjust as needed)
numeric_cols = [ 'batting_position', 'runs','balls', 'fours', 'sixes', 'strike_rate',
                'overs', 'total_balls', 'dots', 'maidens',
                'conceded', 'fours_conceded', 'sixes_conceded',
                'wickets', 'LBW', 'Bowled', 'noballs',
                'wides', 'economy_rate', 'catches', 'stumping', 'direct_hit', 'indirect_hit']

df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
len(numeric_cols)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Normalize the numeric columns
scaler = MinMaxScaler()
df_normalized = df.copy()
df_normalized[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [ ]:
# Correlation Matrix
plt.figure(figsize=(15, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

In [ ]:


selected_features = ['match_id'] + numeric_cols + ['strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp']
df_selected = df_normalized[selected_features]
print("\nSelected features for further analysis/modeling:")
df_selected.head()

In [ ]:
df_selected.to_csv('cleaned_4a8a2e3b.csv', index=False)
print("\nCleaned data saved to 'cleaned_data.csv'")

In [ ]:
## WEATHER FECTHING

In [ ]:
!pip install openmeteo_requests
!pip install retry-requests
!pip install requests_cache

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import time
from tqdm import tqdm  # Import tqdm for progress bar

In [ ]:
# Load match data
matches_df = pd.read_csv("/content/updated_matches_info1.csv")

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Weather API URL
url = "https://archive-api.open-meteo.com/v1/archive"

# List to store weather data
weather_data = []

# Use tqdm to show progress over the DataFrame rows
for index, row in tqdm(matches_df.iterrows(), total=matches_df.shape[0], desc="Processing matches"):
    match_id = row["match_id"]
    latitude = row["venue_lat"]
    longitude = row["venue_long"]
    match_date = row["match_date"]

    # Skip rows with missing lat/lon
    if pd.isna(latitude) or pd.isna(longitude):
        continue

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": match_date,
        "end_date": match_date,
        "daily": "apparent_temperature_mean",
        "hourly": ["relative_humidity_2m", "apparent_temperature", "precipitation", "is_day",
                   "temperature_2m_spread", "temperature_2m"]
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        # Process daily data
        daily = response.Daily()
        daily_apparent_temperature_mean = daily.Variables(0).ValuesAsNumpy()[0]

        # Process hourly data
        hourly = response.Hourly()
        hourly_relative_humidity_2m = hourly.Variables(0).ValuesAsNumpy().mean()
        hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy().mean()
        hourly_precipitation = hourly.Variables(2).ValuesAsNumpy().sum()
        hourly_is_day = hourly.Variables(3).ValuesAsNumpy().mean()
        hourly_temperature_2m_spread = hourly.Variables(4).ValuesAsNumpy().mean()
        hourly_temperature_2m = hourly.Variables(5).ValuesAsNumpy().mean()

        weather_data.append([
            match_id, match_date, latitude, longitude, daily_apparent_temperature_mean,
            hourly_relative_humidity_2m, hourly_apparent_temperature,
            hourly_precipitation, hourly_is_day, hourly_temperature_2m_spread,
            hourly_temperature_2m
        ])

        # Delay to avoid hitting API limits
        time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for match {match_id}: {e}")

# Convert to DataFrame and save
weather_df = pd.DataFrame(weather_data, columns=[
    "match_id", "match_date", "latitude", "longitude", "daily_apparent_temperature_mean",
    "hourly_relative_humidity_2m", "hourly_apparent_temperature",
    "hourly_precipitation", "hourly_is_day", "hourly_temperature_2m_spread",
    "hourly_temperature_2m"
])

weather_df.to_csv("weather.csv", index=False)
print("Weather data saved to weather.csv")

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('cleaned_universal_player.csv')

# Display the DataFrame
print("Original DataFrame:")
df.head()  # Show the first few rows


In [ ]:
df[['batting', 'bowling', 'all_rounder']] = df[['batting', 'bowling', 'all_rounder']].astype(int)

df.head()

In [ ]:
## Concatation of the Input Embedding

#### Create the DataLoader

In [ ]:
# import os
# import numpy as np
# import pandas as pd
# from torch.utils.data import Dataset

# class PlayerMatchDataset(Dataset):
#     """
#     Dataset for autoregressive next-match prediction.

#     For each sample:
#       1. Given a player_id, retrieve universal player features from universal_player.csv.
#          (We drop the 'player_id' and 'cricinfo_id' from the input features.)
#       2. Retrieve all matches for that player from player_matches.csv (sorted by date).
#          The performance columns include:
#            batting_position, runs, balls, fours, sixes, strike_rate, overs, total_balls, dots,
#            maidens, conceded, fours_conceded, sixes_conceded, wickets, LBW, Bowled, noballs,
#            wides, economy_rate, catches, stumping, direct_hit, indirect_hit, strike_rate_fp,
#            batting_fp, bowling_fp, fielding_fp, total_fp.
#          We drop 'match_id' and the fantasy-breakdown columns
#          ('strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp') when forming the input vector.
#       3. Randomly sample a contiguous window of (context_len + 1) matches from this player.
#          The first context_len matches serve as input and matches 2...context_len+1 yield the target fantasy scores.
#       4. For each match in the window:
#          - Load the corresponding match_players CSV from the match_players folder (file: '{match_id}.csv').
#          - Separate the player_ids into team1 and team2 (based on the 'Team' column).
#          - Retrieve their universal features from universal_player.csv.
#          - Load the match info from the match_info folder (file: '{match_id}_info.csv').
#       5. Return a dictionary containing:
#          - 'univ_features': the player's universal features.
#          - 'context_matches': an array of performance features for the context matches.
#          - 'target_scores': an array of target fantasy scores.
#          - 'team1_players': list of team1 players' universal features for each match.
#          - 'team2_players': list of team2 players' universal features for each match.
#          - 'match_info': list of match info dictionaries for each match.
#     """

#     def __init__(self, universal_player_csv, player_matches_csv, match_players_dir, match_info_dir, context_len=5, transform=None):
#         """
#         Args:
#             universal_player_csv (str): Path to universal_player.csv.
#             player_matches_csv (str): Path to player_matches.csv.
#             match_players_dir (str): Directory containing match_players CSV files.
#             match_info_dir (str): Directory containing match info CSV files.
#             context_len (int): Number of context matches to use as input (the target will be matches 2 ... context_len+1).
#             transform (callable, optional): Optional transform to be applied on a sample.
#         """
#         self.context_len = context_len
#         self.transform = transform

#         # Load universal player features and set player_id as index.
#         self.univ_df = pd.read_csv(universal_player_csv)
#         self.univ_df = self.univ_df.set_index('player_id')
#         # Drop cricinfo_id since it is redundant.
#         self.univ_features = self.univ_df.drop(columns=['cricinfo_id'], errors='ignore')

#         # Load player matches and sort by date.
#         self.matches_df = pd.read_csv(player_matches_csv)
#         self.matches_df['date'] = pd.to_datetime(self.matches_df['date'])
#         self.matches_df = self.matches_df.sort_values(by='date')

#         # Build an index: for each player_id, list the row indices in matches_df.
#         self.player_match_indices = {}
#         for player_id, group in self.matches_df.groupby('player_id'):
#             indices = group.index.tolist()
#             # Only include players with at least (context_len + 1) matches.
#             if len(indices) >= (self.context_len + 1):
#                 self.player_match_indices[player_id] = indices

#         self.match_players_dir = match_players_dir
#         self.match_info_dir = match_info_dir

#         # Valid player_ids (those with enough matches)
#         self.player_ids = list(self.player_match_indices.keys())

#     def __len__(self):
#         return len(self.player_ids)

#     def __getitem__(self, idx):
#         """
#         Returns a dictionary with:
#         - 'player_id': the player's id.
#         - 'univ_features': universal features for the player.
#         - 'context_matches': performance features for the first context_len matches.
#         - 'target_scores': fantasy scores from matches 2 to context_len+1.
#         - 'team1_players': for each match, universal features for team1 players.
#         - 'team2_players': for each match, universal features for team2 players.
#         - 'match_info': match-specific info for each match.
#         """
#         # Select the player
#         player_id = self.player_ids[idx]
#         # Get the player's universal features (as a numpy array of floats)
#         player_univ = self.univ_features.loc[player_id].values.astype(float)
#         print(player_univ)
        
#         # Get indices for all matches of this player.
#         match_indices = self.player_match_indices[player_id]
#         # Randomly select a contiguous window of (context_len + 1) matches.
#         start_idx = np.random.randint(0, len(match_indices) - self.context_len)
#         selected_indices = match_indices[start_idx : start_idx + self.context_len + 1]
#         selected_matches = self.matches_df.loc[selected_indices]

#         # For input, we drop the columns that are not used: 'match_id' and fantasy-breakdown columns.
#         exclude_cols = ['teamname', 'match_id', 'strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp']
#         # Target fantasy scores (assumed to be in column 'total_fp') from matches 2 ... context_len+1.
#         target_scores = selected_matches.iloc[1:self.context_len+1]['total_fp'].values.astype(float)
#         context_matches = selected_matches.iloc[:self.context_len].drop(columns=exclude_cols, errors='ignore')

#         # For each match in the window, retrieve team player ids and match info.
#         team1_players_list = [] 
#         team2_players_list = []
#         match_info_list = []
#         for _, match in selected_matches.iterrows():
#             match_id = match['match_id']
#             # Load match players file (expects a file named "{match_id}.csv")
#             match_players_file = os.path.join(self.match_players_dir, f"{match_id}.csv")
#             match_players_df = pd.read_csv(match_players_file)
            
#             # Determine the player's team for this match.
#             player_team_series = match_players_df[match_players_df['player_id'] == player_id]['Team']
#             if not player_team_series.empty:
#                 player_team = player_team_series.iloc[0]
#             else:
#                 # If player's team is not found, default to first team (or handle as needed).
#                 player_team = match_players_df.iloc[0]['Team']
            
#             # Use the player's team name to split the teams.
#             team1_ids = match_players_df[match_players_df['Team'] == player_team]['player_id'].tolist()
#             team2_ids = match_players_df[match_players_df['Team'] != player_team]['player_id'].tolist()

#             # Retrieve universal features for these players.
#             team1_features = self.univ_features.reindex(team1_ids).dropna().values.astype(float)
#             team2_features = self.univ_features.reindex(team2_ids).dropna().values.astype(float)

#             team1_players_list.append(team1_features)
#             print(team1_players_list.size())
#             team2_players_list.append(team2_features)
#             print(len(team2_players_list))
            
#             # Load match info file (expects a file named "{match_id}_info.csv")
#             match_info_file = os.path.join(self.match_info_dir, f"{match_id}_info.csv")
#             match_info_df = pd.read_csv(match_info_file)
#             match_info_list.append(match_info_df.to_dict(orient='list'))
#             print(len(match_info_list))

#         sample = {
#             'player_id': player_id,
#             'univ_features': player_univ,                         # Universal features for the player.
#             'context_matches': context_matches.values.astype(float),  # Performance features for context matches.
#             'target_scores': target_scores,                       # Target fantasy scores.
#             'team1_players': team1_players_list,                  # List (per match) of team1 players' universal features.
#             'team2_players': team2_players_list,                  # List (per match) of team2 players' universal features.
#             'match_info': match_info_list                         # List (per match) of match info dictionaries.
#         }

#         if self.transform:
#             sample = self.transform(sample)
#         return sample


In [8]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

class PlayerMatchDataset(Dataset):
    """
    Dataset for autoregressive next-match prediction.

    For each sample:
      1. Given a player_id, retrieve universal player features from universal_player.csv.
         (The 'player_id' and 'cricinfo_id' columns are dropped from the input features.)
      2. Retrieve all matches for that player from a directory of player match files
         (each file is named "{player_id}.csv" and contains the matches for that player,
         already sorted by date).
         The performance columns include:
           batting_position, runs, balls, fours, sixes, strike_rate, overs, total_balls, dots,
           maidens, conceded, fours_conceded, sixes_conceded, wickets, LBW, Bowled, noballs,
           wides, economy_rate, catches, stumping, direct_hit, indirect_hit, strike_rate_fp,
           batting_fp, bowling_fp, fielding_fp, total_fp.
         We drop 'match_id' and the fantasy-breakdown columns
         ('strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp') when forming the input vector.
      3. Randomly sample a contiguous window of (context_len + 1) matches for this player.
         The first context_len matches serve as input and matches 2 ... context_len+1 yield the target fantasy scores.
      4. For each match in the window:
         - Load the corresponding match players CSV from the match_players folder (file: '{match_id}.csv').
         - Determine the player's team for that match and then separate player_ids into:
              team1_ids: those belonging to the same team as the player,
              team2_ids: those belonging to the other team.
         - Retrieve their universal features from universal_player.csv.
         - Retrieve match info from a single match_info CSV (by matching on match_id).
      5. Return a dictionary containing:
         - 'player_id': the player's id.
         - 'univ_features': the player's universal features.
         - 'context_matches': a numpy array of performance features for the context matches.
         - 'target_scores': a numpy array of target fantasy scores (for matches 2 ... context_len+1).
         - 'team1_players': list (per match) of team1 players' universal features.
         - 'team2_players': list (per match) of team2 players' universal features.
         - 'match_info': list (per match) of match info dictionaries.
    """

    def __init__(self, universal_player_csv, player_matches_dir, match_players_dir, match_info_csv, context_len=25, transform=None):
        """
        Args:
            universal_player_csv (str): Path to universal_player.csv.
            player_matches_dir (str): Directory containing CSV files for each player's matches (named '{player_id}.csv').
            match_players_dir (str): Directory containing match_players CSV files.
            match_info_csv (str): Path to the CSV file containing match info for all matches.
            context_len (int): Number of context matches to use as input 
                                (target will be matches 2 ... context_len+1).
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.context_len = context_len
        self.transform = transform

        # Load universal player features and set player_id as index.
        self.univ_df = pd.read_csv(universal_player_csv)
        self.univ_df = self.univ_df.set_index('player_id')
        # Drop 'cricinfo_id' as it is redundant.
        self.univ_features = self.univ_df.drop(columns=['cricinfo_id'], errors='ignore')

        # Instead of a single file for player matches, we assume a directory where each player's
        # matches are stored in a file named '{player_id}.csv'. We build a list of valid player_ids
        # by checking which player match files exist and ensuring they have at least (context_len+1) rows.
        self.player_matches_dir = player_matches_dir
        self.player_ids = []
        self.player_match_data = {}  # Key: player_id, Value: DataFrame of that player's matches.
        for player_id in self.univ_features.index:
            match_file = os.path.join(player_matches_dir, f"{player_id}.csv")
            if os.path.exists(match_file):
                df_matches = pd.read_csv(match_file)
                # Assume the matches in this file are already sorted by date.
                if len(df_matches) >= (self.context_len + 1):
                    self.player_ids.append(player_id)
                    self.player_match_data[player_id] = df_matches

        self.match_players_dir = match_players_dir

        # Load the single match_info CSV and set match_id as index for fast lookup.
        self.match_info_df = pd.read_csv(match_info_csv)
        self.match_info_df = self.match_info_df.set_index('match_id')
        # Optionally drop columns not needed.
        self.match_info_df = self.match_info_df.drop(columns=['team1', 'team2', 'toss_winner', "toss_decision", "winner"], errors='ignore')
        #Todo don't just drop them we need to infer from them in sense of player's team ->winner 1 else 0

    def __len__(self):
        return len(self.player_ids)

    def __getitem__(self, idx):
        """
        Returns a dictionary with:
          - 'player_id': the player's id.
          - 'univ_features': universal features for the player.
          - 'context_matches': a numpy array of performance features for the context matches.
          - 'target_scores': a numpy array of target fantasy scores (for matches 2 ... context_len+1).
          - 'team1_players': list (per match) of team1 players' universal features.
          - 'team2_players': list (per match) of team2 players' universal features.
          - 'match_info': list (per match) of match info dictionaries.
        """
        #ToDO not just total_fantasy score, rather a weighted sum of error for batting_bowling fp's, fielding_fp with lesser weightage to the overall total_fp  
        # Select the player.
        player_id = self.player_ids[idx]
        # Retrieve the player's universal features as a numpy array.
        player_univ = self.univ_features.loc[player_id].values.astype(float)
        
        # Load this player's matches DataFrame from the pre-loaded dictionary.
        df_matches = self.player_match_data[player_id]
        total_matches = len(df_matches)
        # Randomly select a contiguous window of (context_len + 1) matches.
        start_idx = np.random.randint(1, total_matches - self.context_len+1)
        window_df = df_matches.iloc[start_idx-1 : start_idx + self.context_len]

        # For input, drop the unused columns.
        exclude_cols = ['teamname', 'match_id',
                        'strike_rate_fp', 'batting_fp', 'bowling_fp', 'fielding_fp', 'total_fp']
        # Extract target fantasy scores (from column 'total_fp') for matches 2 ... context_len+1.
        target_scores = window_df.iloc[1:self.context_len+1]['total_fp'].values.astype(float)
        #Todo account for all sub_fp's (stage -2)
        # Extract performance features for input matches.
        context_matches = window_df.iloc[:self.context_len].drop(columns=exclude_cols, errors='ignore')

        # For each match in the window, retrieve team players and match info.
        team1_players_list = [] 
        team2_players_list = []
        match_info_list = []
        for _, match in window_df.iterrows():
            match_id = match['match_id']
            # Load match players file (expects a file named "{match_id}.csv").
            match_players_file = os.path.join(self.match_players_dir, f"{int(match_id)}.csv")
            match_players_df = pd.read_csv(match_players_file)
            
            # Determine the team of the sample's player in this match.
            player_team_series = match_players_df[
                match_players_df['player_id'] == player_id]['Team']
            #? rather we can refer it from window_df, there in the excluded_cols same as match['team_name']
            if not player_team_series.empty:
                player_team = player_team_series.iloc[0]
            else:
                player_team = match_players_df.iloc[0]['Team'] #?why's this, though it won't execute
            
            # Split player_ids into two groups based on the player's team.
            team1_ids = match_players_df[
                match_players_df['Team'] == player_team]['player_id'].tolist()
            #! we just need the team2 player_ids (we are omitting the intra-team interactions...)
            team2_ids = match_players_df[
                match_players_df['Team'] != player_team]['player_id'].tolist()

            # Retrieve universal features for these players.
            team1_features = self.univ_features.reindex(team1_ids).dropna().values.astype(float)
            team2_features = self.univ_features.reindex(team2_ids).dropna().values.astype(float)
            team1_players_list.append(team1_features)
            team2_players_list.append(team2_features)
            
            # Retrieve match info using match_id from the single match_info DataFrame.
            if match_id in self.match_info_df.index:
                match_info_dict = self.match_info_df.loc[match_id].to_dict()
            else:
                match_info_dict = { }
            match_info_list.append(match_info_dict)
            #? wht's this 

        # --- START OF CHANGES: convert to properly shaped torch.Tensors ---
        import torch

        # universal features
        univ_features = torch.tensor(player_univ, dtype=torch.float32)  # (feat_dim,)

        # context matches
        context_matches = torch.tensor(context_matches.values.astype(float),
                                       dtype=torch.float32)  # (context_len, perf_dim)

        # target scores
        target_scores = torch.tensor(target_scores, dtype=torch.float32)  # (context_len,)

        # team1 players: pad to max players across the window, then stack
        max1 = max(arr.shape[0] for arr in team1_players_list)
        feat_dim = team1_players_list[0].shape[1] if max1>0 else 0
        padded1 = [
            np.pad(arr, ((0, max1 - arr.shape[0]), (0, 0)), mode='constant')
            for arr in team1_players_list
        ]
        team1_players = torch.tensor(np.stack(padded1), dtype=torch.float32)
        # shape: (context_len+1, max1, feat_dim)

        # team2 players: same
        max2 = max(arr.shape[0] for arr in team2_players_list)
        feat_dim2 = team2_players_list[0].shape[1] if max2>0 else 0
        padded2 = [
            np.pad(arr, ((0, max2 - arr.shape[0]), (0, 0)), mode='constant')
            for arr in team2_players_list
        ]
        team2_players = torch.tensor(np.stack(padded2), dtype=torch.float32)
        # shape: (context_len+1, max2, feat_dim2)
        #? wht's this

        # match_info: convert list of dicts to array in fixed key order
        keys = list(self.match_info_df.columns)
        info_arr = np.stack([[d.get(k, 0.0) for k in keys] for d in match_info_list])
        match_info = torch.tensor(info_arr, dtype=torch.float32)
        # shape: (context_len+1, len(keys))

        # --- END OF CHANGES ---

        sample = {
            'player_id': player_id,
            'univ_features': univ_features,
            'context_matches': context_matches,
            'target_scores': target_scores,
            'team1_players': team1_players[:, :11, :],
            'team2_players': team2_players[:, :11, :],
            'match_info': match_info
        }
        
        if self.transform:
            sample = self.transform(sample)
        return sample


In [9]:
import os
from torch.utils.data import DataLoader

# Define file paths (update these paths as needed for your folder structure)
universal_player_csv = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\cleaned_universal_player.csv'
player_matches_dir = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\Processed_Player_records'  # Contains files named like {player_id}.csv (each containing that player's matches)
match_players_dir = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\processed_GlobalMatchrecords'    # Contains files like {match_id}.csv
match_info_csv = r'C:\Users\kumar\IPL_Fantasy_Score_Prediction\Ashu\Test_1\cleaned_matchinfo_without_venue_with_updated_match_number.csv'               # Single CSV containing all match info

# Define the context length (number of matches to use as context)
# For example, if config.CONTEXT_LEN is defined in your config module:
# config.CONTEXT_LEN = 5

# Initialize the dataset
dataset = PlayerMatchDataset(
    universal_player_csv=universal_player_csv,
    player_matches_dir=player_matches_dir,  # This parameter may be ignored if you use the directory version
    match_players_dir=match_players_dir,
    match_info_csv=match_info_csv,
    context_len=config.CONTEXT_LEN
)

# def get_shape(lst):
#     shape = []
#     while isinstance(lst, list):
#         shape.append(len(lst))
#         if len(lst) == 0:
#             break
#         lst = lst[0]
#     return tuple(shape)



# Initialize the DataLoader
dataloader = DataLoader(dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=0)

# # Iterate through one batch to check the output shapes and values
# for batch in dataloader:
#     print("Player IDs:", batch['player_id'])  # list of player_ids (length = batch_size)
    
#     print("Universal features shape:", (batch['univ_features']).shape)  # e.g., (batch_size, num_features)
    
#     print("Context matches shape:", batch['context_matches'].shape)  # e.g., (batch_size, context_len, performance_feature_dim)
    
#     print("Target scores shape:", batch['target_scores'])      # e.g., (batch_size, context_len)
    
#     # The following are lists of length (context_len+1); each element is a numpy array.
#     # team1_players_tensor = list_of_tensors_to_3d(batch['team1_players'])
#     # team1_players_tensor = team1_players_tensor.squeeze(2)
#     print("Number of matches in team1_players (per sample):", batch['team1_players'].shape)
    
#     # team2_players_tensor = list_of_tensors_to_3d(batch['team2_players'])
#     # team2_players_tensor = team2_players_tensor.squeeze(2)
#     print("Number of matches in team2_players (per sample):", batch['team2_players'].shape)
    
#     # match_info_tensor = list_of_dicts_to_tensor(batch['match_info'])
#     print("Number of matches in match_info (per sample):", (batch['match_info']).shape)
#     break


In [10]:
len(dataset)

2354

In [12]:
for batch in dataloader:
    print("Player IDs:", batch['player_id'])  # list of player_ids (length = batch_size)
    
    print("Universal features shape:", (batch['univ_features']).shape)  # e.g., (batch_size, num_features)
    
    print("Context matches shape:", batch['context_matches'].shape)  # e.g., (batch_size, context_len, performance_feature_dim)
    
    print("Target scores shape:", batch['target_scores'].shape)      # e.g., (batch_size, context_len)
    
    # The following are lists of length (context_len+1); each element is a numpy array.
    # team1_players_tensor = list_of_tensors_to_3d(batch['team1_players'])
    # team1_players_tensor = team1_players_tensor.squeeze(2)
    print("Number of matches in team1_players (per sample):", batch['team1_players'].shape)
    
    # team2_players_tensor = list_of_tensors_to_3d(batch['team2_players'])
    # team2_players_tensor = team2_players_tensor.squeeze(2)
    print("Number of matches in team2_players (per sample):", batch['team2_players'].shape)
    break
    

Player IDs: ['1f28fdfd', 'a756e61a', 'eb0f2802', '07c8a2f4', 'c9c0fe50', '498aacac', '7d15170b', 'ea3ffd1b', 'ee1b6c27', 'c41043e6', '99b75528', 'c63671e9', '45ca12a7', '0ea1455a', 'dca8273d', 'fd96de7c', '025a092b', '9c7d7cda', '6a71ba3a', '6aed7e79', '9f701298', '42172c3c', '1747ea18', 'bb18be76', '4312e0fd', '6a685375', '39e9f0c9', '616a8149', '39ed0d2f', '85d8110a', '5b3acc53', 'c96311d4', 'd76ab352', '7c7d63a2', '69d03465', '7c503806', '2baad3bd', '1efb8a28', 'f56cd097', 'f3ca5af8', 'ccc04a61', '5b7ab5a9', '1798078c', 'b552a935', '1786a21b', '1c5c3dce', '4663bd23', 'a0d66374', 'b91083bd', '3957a70a', 'c31adb5c', '93fabc71', 'ade90de7', '85b3fab2', '8db7f47f', '5d60560f', '51a60c16', '97b0e4ff', '779b7f4d', '6cc4d322', 'b52ffbbd', 'cb461421', 'd7a57f75', '47b4ad09', '28131839', '0a67aec0', '1a156c88', '3812d56b', 'a83c44b8', '23638956', 'dc4686e6', '3a7ea906', '59792462', '57590dbb', 'a343262c', '14157abf', '2498e163', 'b8704508', '9ed95736', 'cdf59953', '11614d87', '7a8bd078', '70

In [ ]:
def list_of_tensors_to_3d(tensor_list):
    """
    Converts a list of tensors into a 3D tensor with shape:
    (1, number_of_tensors, *inner_tensor_shape)
    
    Args:
        tensor_list (list of torch.Tensor): List of tensors with identical shapes.
        
    Returns:
        torch.Tensor: A tensor with the new shape (1, len(tensor_list), inner dims...).
    """
    # First stack the tensors along dimension 0: shape becomes (number_of_tensors, inner dims...)
    stacked = torch.stack(tensor_list, dim=0)
    # Add a leading dimension to obtain the final shape (1, number_of_tensors, inner dims...)
    return stacked.unsqueeze(0)


def list_of_dicts_to_tensor(data, key_order=None):
    """
    Convert a list of dictionaries (each with tensor or numeric values) 
    into a 3D tensor of shape (1, number_of_dicts, number_of_keys).

    Args:
        data (list): List of dictionaries where each dictionary contains the same keys.
        key_order (list, optional): Specific order of keys to extract from each dictionary.
                                    If None, keys from the first dictionary are used.

    Returns:
        torch.Tensor: A 3D tensor with shape (1, len(data), len(key_order)).
    """
    if not data:
        raise ValueError("The input data list is empty.")
    
    # Use keys from the first dictionary if no order is specified.
    if key_order is None:
        key_order = list(data[0].keys())
    
    values_list = []
    for d in data:
        # Extract values in the specified order. Convert tensor values to scalar if necessary.
        values = []
        for key in key_order:
            value = d[key]
            if isinstance(value, torch.Tensor):
                # Assuming tensor is of shape (1,)
                values.append(value.item())
            else:
                values.append(value)
        values_list.append(values)
    
    # Convert the list of lists to a 2D tensor.
    tensor_2d = torch.tensor(values_list)
    # Add a new dimension at the beginning to make it 3D.
    tensor_3d = tensor_2d.unsqueeze(0)
    return tensor_3d

In [ ]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\umbar\OneDrive\Documents\Fantasy Point Prediction\IPL_Fantasy_Score_Prediction\Ashu


## Model Architecture

In [7]:
class PlayerEmbedding(nn.Module):
  """ HEre we are doing the Proj of the raw Player embedding into the PERFORMANCE_EMBD_DIM """
  def __init__(self, in_channels=config.PLAYER_INPUT_DIM, out_channels=config.PERFORMANCE_EMBD_DIM):
    super().__init__()
    self.proj = nn.Linear(in_channels, out_channels)

  def forward(self, x):
    # print(f"In Player Embd  {x.shape}")
    # x: (B, PLAYER_INPUT_DIM) or flattened (B*T, PLAYER_INPUT_DIM)
    return self.proj(x)   ## ( B/B*T, PERFORMANCE_EMBD)

In [20]:
class MatchEmbedding(nn.Module):
    """
    Computes a match-level embedding from team and match information.
    Expected input shapes (for T matches):
      team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
      team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
      match_info: (B, T, match_info_dim)
    Output:
      (B, T, PERFORMANCE_EMBD_DIM)
    """
    def __init__(self, player_embedding_module, in_channels=(2*config.PERFORMANCE_EMBD_DIM + config.MATCH_INPUT_EMBD), out_channels=config.PERFORMANCE_EMBD_DIM):
        super().__init__()
        self.player = player_embedding_module
        self.proj = nn.Linear(in_channels, out_channels)

    def forward(self, team1_players, team2_players, match_info):
        """
          1. Get player embeddings using self.player.
          2. Sum (or pool) embeddings for each team.
          3. Concatenate team representations with match_info.
          4. Project the concatenated vector to obtain the final match embedding.
        """
        B, T, num_team1, _ = team1_players.shape
        # print(f"In match embd {team1_players.shape} & {match_info.shape}, B {B}, T, {T}, nums_team1 {num_team1}")
        # Compute player embeddings
        team1_flat = team1_players.reshape(B * T, num_team1, -1)  # (B*T, num_team1, PLAYER_INPUT_DIM)
        team1_embeds = self.player(team1_flat)  # (B*T, num_team1, PERFORMANCE_EMBD_DIM)
        team1_sum = team1_embeds.sum(dim=1)  # (B*T, PERFORMANCE_EMBD_DIM)
        team1_sum = team1_sum.reshape(B, T, -1)  # (B, T, PERFORMANCE_EMBD_DIM)

        B, T, num_team2, _ = team2_players.shape
        team2_flat = team2_players.reshape(B * T, num_team2, -1)
        team2_embeds = self.player(team2_flat)  # (B*T, num_team2, PERFORMANCE_EMBD_DIM)
        team2_sum = team2_embeds.sum(dim=1)  # (B*T, PERFORMANCE_EMBD_DIM)
        team2_sum = team2_sum.reshape(B, T, -1)  # (B, T, PERFORMANCE_EMBD_DIM)

        # Concatenate team summaries with match-level info along last dimension.
        # match_info: (B, T, match_info_dim)
        fused = torch.cat([team1_sum, team2_sum, match_info], dim=-1)  # (B, T, 2*PERFORMANCE_EMBD_DIM + match_info_dim)
        match_embedding = self.proj(fused)  # (B, T, PERFORMANCE_EMBD_DIM)

        return match_embedding



In [ ]:
class PerformanceEmbedding(nn.Module):
    """
    Combines a sequence of player embeddings with a sequence of match embeddings to produce a sequence
    of performance embeddings. For each time step:
      1. Obtain the player's embedding from player_input (B, T, PLAYER_INPUT_DIM).
      2. Obtain the match embedding using the MatchEmbedding module, which now expects team-level
         inputs with a time dimension (B, T, ...).
      3. Concatenate these embeddings (+moresulting in a vector of dimension 2 * PERFORMANCE_EMBD_DIM).
      4. Project the concatenated vector to PERFORMANCE_EMBD_DIM.

    Expected input shapes:
      player_input: (B, T, PLAYER_INPUT_DIM)
      team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
      team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
      match_info: (B, T, match_info_dim)

    Output:
      (B, T, PERFORMANCE_EMBD_DIM)
    """
    def __init__(self, player_embedding_module, match_embedding_module, out_channels=config.PERFORMANCE_EMBD_DIM):
        super().__init__()
        self.player_embedding_module = player_embedding_module
        self.match_embedding_module = match_embedding_module
        self.performance_proj = nn.Linear(config.PERFORMANCE_INPUT_DIM, out_channels)
        # Linear layer to map concatenated [player_emb; match_emb] (dimension 2*PERFORMANCE_EMBD_DIM)
        # to PERFORMANCE_EMBD_DIM.
        self.proj = nn.Linear(3 * config.PERFORMANCE_EMBD_DIM, out_channels)

    def forward(self, player_input, player_performance_input, team1_players, team2_players, match_info):
        """
        player_input: (B, T, PLAYER_INPUT_DIM) - raw features for a specific player across T matches.
        team1_players: (B, T, num_team1, PLAYER_INPUT_DIM) - raw features for team1 players per match.
        team2_players: (B, T, num_team2, PLAYER_INPUT_DIM) - raw features for team2 players per match.
        match_info: (B, T, match_info_dim) - extra normalized match information per match.
        """
        B, T, _ = player_performance_input.shape
        # print(f"In performance embedding {player_input.shape} && {player_performance_input.shape}")
        # Compute player's embedding for each match time step.
        # Reshape to (B*T, PLAYER_INPUT_DIM) so that the player_embedding_module can be applied, then reshape back.
        player_emb = self.player_embedding_module(player_input.reshape(B, -1))  # (B, PERFORMANCE_EMBD_DIM)
        player_emb = player_emb.unsqueeze(1).repeat(1, T, 1)  # (B, T, PERFORMANCE_EMBD_DIM)
        #!why we are repeating this.....
        player_performance_emb = self.performance_proj(player_performance_input.reshape(B * T, -1))  # (B*T, PERFORMANCE_INPUT_DIM) => (B*T, PERFORMANCE_EMBD_DIM)
        player_performance_emb = player_performance_emb.reshape(B, T, -1)  # (B, T, PERFORMANCE_EMBD_DIM)

        # Compute match embedding across T time steps.
        # Ensure that the match_embedding_module expects inputs with a time dimension.
        match_emb = self.match_embedding_module(team1_players, team2_players, match_info)  # (B, T, PERFORMANCE_EMBD_DIM)

        # Concatenate the player's embedding and match embedding for each time step.
        combined = torch.cat([player_emb, player_performance_emb, match_emb], dim=-1)  # (B, T, 3 * PERFORMANCE_EMBD_DIM)
        #! what is player_emb, and player_performance_emb (diffence....)
        # Project the concatenated vector back to PERFORMANCE_EMBD_DIM.
        performance_emb = self.proj(combined)  # (B, T, PERFORMANCE_EMBD_DIM)
        #! performance_proj -> performance_embd_dim  and the combined with the match_ifo is also project to the performance_embd_dim 
        #TODO should we use another higher dimesion 
        return performance_emb


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, n_heads, d_embed, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        # This combines the Wq, Wk and Wv matrices into one matrix
        self.in_proj = nn.Linear(d_embed, 3 * d_embed, bias=in_proj_bias)
        # This one represents the Wo matrix
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
        self.n_heads = n_heads   ## how many heads u want ?
        self.d_head = d_embed // n_heads   ## the original embedding get divided in the all heads equally
   

    def forward(self, x, causal_mask=False):

        # x: # (Batch_Size, Seq_Len, Dim)

        # (Batch_Size, Seq_Len, Dim)
        input_shape = x.shape

        # (Batch_Size, Seq_Len, Dim)
        batch_size, sequence_length, d_embed = input_shape

        # (Batch_Size, Seq_Len, H, Dim / H)
        qkv_shape = (batch_size, sequence_length, self.n_heads, self.d_head)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim * 3) -> 3 tensor of shape (Batch_Size, Seq_Len, Dim)
        q, k, v = self.in_proj(x).chunk(3, dim=-1)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, H, Dim / H) -> (Batch_Size, H, Seq_Len, Dim / H)
        q = q.view(qkv_shape).transpose(1, 2)
        k = k.view(qkv_shape).transpose(1, 2)
        v = v.view(qkv_shape).transpose(1, 2)

        # (Batch_Size, H, Seq_Len, Dim / H) @ (Batch_Size, H, Dim / H, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight = q @ k.transpose(-1, -2)

        if causal_mask:
            # Mask where the upper triangle (above the principal diagonal) is 1
            mask = torch.ones_like(weight, dtype=torch.bool).triu(1)
            # Fill the upper triangle with -inf
            weight.masked_fill_(mask, -torch.inf)

        # Divide by d_k (Dim / H).
        # (Batch_Size, H, Seq_Len, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight /= math.sqrt(self.d_head)

        # (Batch_Size, H, Seq_Len, Seq_Len) -> (Batch_Size, H, Seq_Len, Seq_Len)
        weight = F.softmax(weight, dim=-1)

        # (Batch_Size, H, Seq_Len, Seq_Len) @ (Batch_Size, H, Seq_Len, Dim / H) -> (Batch_Size, H, Seq_Len, Dim / H)
        output = weight @ v

        # (Batch_Size, H, Seq_Len, Dim / H) -> (Batch_Size, Seq_Len, H, Dim / H)
        output = output.transpose(1, 2)

        # (Batch_Size, Seq_Len, H, Dim / H) -> (Batch_Size, Seq_Len, Dim)
        output = output.reshape(input_shape)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        output = self.out_proj(output)

        # (Batch_Size, Seq_Len, Dim)
        return output


In [10]:
class a_layer(nn.Module):
    """ A Single Transformer Layer/Block """

    def __init__(self, n_head: int, n_embd: int):
        super().__init__()
        # Pre-attention norm
        self.layernorm_1 = nn.LayerNorm(n_embd)
        # Self attention
        self.attention = SelfAttention(n_head, n_embd)
        # Pre-FNN norm
        self.layernorm_2 = nn.LayerNorm(n_embd)
        # Feedforward layer
        self.linear_1 = nn.Linear(n_embd, 4 * n_embd)
        self.linear_2 = nn.Linear(4 * n_embd, n_embd)

    def forward(self, x):
        # (Batch_Size, Seq_Len, Dim)
        residue = x

        ### SELF ATTENTION ###

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.layernorm_1(x)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.attention(x, causal_mask=True)

        # (Batch_Size, Seq_Len, Dim) + (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x += residue

        ### FEEDFORWARD LAYER ###
        # Apply a feedforward layer where the hidden dimension is 4 times the embedding dimension.

        residue = x
        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.layernorm_2(x)

        # (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, 4 * Dim)
        x = self.linear_1(x)

        # (Batch_Size, Seq_Len, 4 * Dim) -> (Batch_Size, Seq_Len, 4 * Dim)
        x = x * torch.sigmoid(1.702 * x)   # QuickGELU activation function found best for this work

        # (Batch_Size, Seq_Len, 4 * Dim) -> (Batch_Size, Seq_Len, Dim)
        x = self.linear_2(x)

        # (Batch_Size, Seq_Len, Dim) + (Batch_Size, Seq_Len, Dim) -> (Batch_Size, Seq_Len, Dim)
        x += residue

        return x

In [11]:
class CrossAttention(nn.Module):
    def __init__(self, d_embed=config.PERFORMANCE_EMBD_DIM, d_cross=config.PERFORMANCE_EMBD_DIM, n_heads=8, in_proj_bias=True, out_proj_bias=True):
        super().__init__()
        # Initialize linear layers for query, key, and value projections.
        # q_proj: projects the query input with shape (batch_size, seq_length_q=1, d_embed)
        self.q_proj   = nn.Linear(d_embed, d_embed, bias=in_proj_bias)
        # k_proj: projects the key input with shape (batch_size, seq_length_kv=11, d_cross)
        self.k_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        # v_proj: projects the value input with shape (batch_size, seq_length_kv=11, d_cross)
        self.v_proj   = nn.Linear(d_cross, d_embed, bias=in_proj_bias)
        # out_proj: projects the concatenated multi-head outputs back to the original embedding dimension.
        self.out_proj = nn.Linear(d_embed, d_embed, bias=out_proj_bias)
        self.n_heads = n_heads                     # Number of attention heads.
        self.d_head = d_embed // n_heads           # Dimensionality per head.

    def forward(self, x, y):
        """
        Compute cross-attention between two sequences.

        Parameters:
            x (Tensor): Query input tensor of shape (batch_size, seq_length_q=1, d_embed)
            y (Tensor): Key/Value input tensor of shape (batch_size, seq_length_kv=11, d_cross)

        Returns:
            Tensor: Output tensor after applying cross-attention, with shape (batch_size, seq_length_q=1, d_embed)
        """
        # Save original input shape.
        input_shape = x.shape  # (batch_size, seq_length_q=1, d_embed)
        batch_size, sequence_length, d_embed = input_shape

        # Define intermediate shape for splitting into multiple heads.
        # New shape: (batch_size, seq_length, n_heads, d_head)
        interim_shape = (batch_size, -1, self.n_heads, self.d_head)

        # Apply linear projections:
        # q: project queries from x.
        q = self.q_proj(x)  # (batch_size, seq_length_q=1, d_embed)
        # k: project keys from y.
        k = self.k_proj(y)  # (batch_size, seq_length_kv=11, d_embed)
        # v: project values from y.
        v = self.v_proj(y)  # (batch_size, seq_length_kv=11, d_embed)

        # Reshape projections to separate attention heads.
        # Transform shape to (batch_size, n_heads, seq_length, d_head)
        q = q.view(interim_shape).transpose(1, 2)
        k = k.view(interim_shape).transpose(1, 2)
        v = v.view(interim_shape).transpose(1, 2)

        # Compute attention scores using scaled dot-product:
        # Multiply queries with transposed keys.
        # Resulting shape: (batch_size, n_heads, seq_length_q=1, seq_length_kv=11)
        weight = q @ k.transpose(-1, -2)

        # Scale the scores by the square root of head dimension to stabilize gradients.
        weight /= math.sqrt(self.d_head)

        # Normalize the attention weights using softmax along the key dimension.
        weight = F.softmax(weight, dim=-1)

        # Compute the weighted sum over the values.
        # Output shape: (batch_size, n_heads, seq_length_q=1, d_head)
        output = weight @ v

        # Reorder dimensions to combine multiple heads:
        # Transpose back to (batch_size, seq_length_q=1, n_heads, d_head) and make contiguous.
        output = output.transpose(1, 2).contiguous()

        # Merge the multi-head outputs to recover original embedding dimension.
        # Final shape: (batch_size, seq_length_q=1, d_embed)
        output = output.view(input_shape)

        # Apply the final output projection.
        output = self.out_proj(output)

        return output  # Return the final cross-attention output.


In [ ]:
class NextFormPredictor(nn.Module):
    """
    Predicts a sequence of performance embeddings autoregressively.
    For loss, for each time step t (from 0 to T-2), the predicted performance embedding at t is used,
    together with the ground-truth player and match embeddings at time t+1, to predict the fantasy score.
    The loss is then computed over all such time steps by flattening the predictions and targets,
    mimicking next-word prediction loss calculation.

    Expected input shapes:
      player_input: (B, PLAYER_INPUT_DIM)
      player_performance_input: (B, T, PLAYER_INPUT_DIM)
      team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
      team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
      match_info: (B, T, match_info_dim)
      target: (B, T) fantasy scores for matches 2..T
    Output:
      perf_emb: (B, T, PERFORMANCE_EMBD_DIM) -- the performance embedding sequence
      loss: scalar loss computed over all predicted fantasy scores
    """
    def __init__(self, player_embedding_module, match_embedding_module, fantasy_score_prediction_module, custom_loss, embedding_dim=config.PERFORMANCE_EMBD_DIM, num_layers=6, n_head=8):
        super().__init__()
        self.player_embedding_module = player_embedding_module
        self.match_embedding = match_embedding_module(player_embedding_module)
        self.token_embedding = PerformanceEmbedding(
            player_embedding_module,
            self.match_embedding
        )       
        # Positional embedding for T tokens (no CLS token)
        # self.cross_attention_layers = nn.ModuleList([CrossAttention(n_heads, d_embed, d_cross) for _ in range(2)])
        self.pos_embedding = nn.Embedding(config.CONTEXT_LEN, embedding_dim)
        self.layers = nn.ModuleList([a_layer(n_head=n_head, n_embd=embedding_dim) for _ in range(num_layers)])
        self.layernorm = nn.LayerNorm(embedding_dim)
        self.out_proj = nn.Linear(embedding_dim, embedding_dim)
        self.fantasy_score_prediction_module = fantasy_score_prediction_module
        self.custom_loss = custom_loss

    def forward(self, player_input, player_performance, team1_players, team2_players, match_info, target=None):
        """
        player_input: (B, PLAYER_INPUT_DIM)
        player_performance: (B, T, PLAYER_PERFORMANCE_INPUT)
        team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
        team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
        match_info: (B, T, match_info_dim)
        target: (B, T-1) fantasy scores for matches 2..T + 1 (if provided) // it should be (B,T) .. 2-->T + 1
        """
        B, T, _ = player_performance.shape  # T = context length
        # team1_players = team1_players[:, :config.CONTEXT_LEN, :11, :]
        # team2_players = team2_players[:, :config.CONTEXT_LEN, :11, :]
        # Obtain performance embeddings sequence (B, T, embedding_dim)
        # print(f"In next form 1 team1 {team1_players[:, :config.CONTEXT_LEN, :, :].shape}, team2 {team2_players[:, :config.CONTEXT_LEN, :, :].shape}, ") 
        
        x = self.token_embedding(player_input, player_performance, team1_players[:, :config.CONTEXT_LEN, :, :], team2_players[:, :config.CONTEXT_LEN, :, :], match_info[:, :config.CONTEXT_LEN, :])


        # Add positional embeddings (using indices 0..T-1)
        pos_ids = torch.arange(T, device=config.DEVICE).unsqueeze(0).expand(B, T)  # (B, T)
        x = x + self.pos_embedding(pos_ids)

        # Process through Transformer layers
        for layer in self.layers:
            x = layer(x)
        x = self.layernorm(x)
        perf_emb = self.out_proj(x)  # (B, T, embedding_dim)

        loss = None
        if target is not None:
            # Shift sequence: for each time step t (0...T-2), use predicted perf_emb at time t to predict
            # the fantasy score for match t+1.
            pred_perf = perf_emb[:, :, :]  # (B, T, embedding_dim)
            Bp, Tp, D = pred_perf.shape
            pred_perf = pred_perf.reshape(B * Tp, D)
            
            player_embd = self.player_embedding_module(player_input)  # (B, 25)
            player_embd = player_embd.unsqueeze(1).repeat(1, Tp, 1)  # (B, T, PERFORMANCE_EMBD_DIM)
            player_embd = player_embd.reshape(B * Tp, D)
            
            
            # Get ground-truth match embedding for match t+1:
            # print(f"In next form 2 team1 {team1_players[:, 1:, :, :].shape}, team2 {team2_players[:, 1:, :, :].shape}") 
            
            target_match_embd = self.match_embedding(
                team1_players[:, 1:, :, :],
                team2_players[:, 1:, :, :],
                match_info[:, 1:, :]
            )  # (B, T-1, embedding_dim)
            target_match_embd = target_match_embd.reshape(B * Tp, D)

            # Predict fantasy scores from the predicted next form, combined with target player and match embeddings.
            # fantasy_score_prediction_module should output shape (B, T-1, 1)
            pred_fantasy = self.fantasy_score_prediction_module(pred_perf, player_embd, target_match_embd)

            # Flatten predictions and targets to compute loss similarly to next-word prediction.
            pred_fantasy_flat = pred_fantasy.reshape(-1)      # (B*(T-1),)
            target_flat = target.reshape(-1)                  # (B*(T-1),)
            loss = self.custom_loss(pred_fantasy_flat, target_flat)
        return perf_emb, loss


In [ ]:
# class NextFormPredictor(nn.Module):
#     """
#     Predicts a sequence of performance embeddings autoregressively.
#     Then, for loss calculation, for each time step t (from 0 to T-2), it uses the predicted
#     performance embedding at time t to predict the fantasy score for match t+1.
#     The prediction is performed by combining:
#       - The predicted performance embedding from time t (acting as the 'next form'),
#       - The ground-truth player embedding for time t+1, and
#       - The ground-truth match embedding for time t+1.
#     These three are passed through an MLP (fantasy_score_prediction_module) to yield the predicted fantasy score.
#     The loss is computed (e.g., via MSE) between these predictions and the provided target fantasy scores.

#     Expected input shapes:



#       player_input: (B, T, PLAYER_INPUT_DIM)

#       team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
#       team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
#       match_info: (B, T, match_info_dim)
#       target: (B, T-1)  (fantasy scores for matches 2..T)
#     Output:
#       performance_embeddings: (B, T, PERFORMANCE_EMBD_DIM)
#       loss: computed fantasy score prediction loss
#     """
#     def __init__(self, player_embedding_module, match_embedding_module, fantasy_score_prediction_module, embedding_dim=config.PERFORMANCE_EMBD_DIM, num_layers=6, n_head=8):
#         super().__init__()
#         self.player_embedding_module = player_embedding_module
#         self.match_embedding_module = match_embedding_module
#         self.token_embedding = PerformanceEmbedding(player_embedding_module, match_embedding_module)
#         self.pos_embedding = nn.Embedding(config.CONTEXT_LEN, embedding_dim)
#         self.layers = nn.ModuleList([a_layer(n_head=n_head, n_embd=embedding_dim) for _ in range(num_layers)])
#         self.layernorm = nn.LayerNorm(embedding_dim)
#         self.out_proj = nn.Linear(embedding_dim, embedding_dim)
#         self.fantasy_score_prediction_module = fantasy_score_prediction_module

#     def forward(self, player_input, team1_players, team2_players, match_info, target=None):
#         """
#         Forward pass.
#           player_input: (B, T, PLAYER_INPUT_DIM)
#           team1_players: (B, T, num_team1, PLAYER_INPUT_DIM)
#           team2_players: (B, T, num_team2, PLAYER_INPUT_DIM)
#           match_info: (B, T, match_info_dim)
#           target: (B, T-1) fantasy scores for matches 2..T (if provided)
#         """
#         B, T, _ = player_input.shape
#         # Obtain a sequence of performance embeddings (B, T, embedding_dim)
#         x = self.token_embedding(player_input, team1_players, team2_players, match_info)
#         # Add positional embeddings: generate indices [0, 1, ..., T-1] and add
#         pos_ids = torch.arange(T, device=player_input.device).unsqueeze(0).expand(B, T)
#         x = x + self.pos_embedding(pos_ids)

#         # Process sequence through transformer layers.
#         for layer in self.layers:
#             x = layer(x)
#         x = self.layernorm(x)
#         # Optionally, project outputs if needed:
#         perf_emb = self.out_proj(x)  # (B, T, embedding_dim)

#         loss = None
#         if target is not None:
#             # For autoregressive fantasy score prediction, we use the performance embedding at time t to predict
#             # the fantasy score for match t+1. Therefore, we shift the sequence:
#             pred_perf = perf_emb[:, :-1, :]  # (B, T-1, embedding_dim) predicted "next form" embeddings.

#             # Get the ground truth player embedding for match t+1:
#             target_player_embd = self.player_embedding_module(player_input[:, 1:, :])  # (B, T-1, embedding_dim)

#             # Get the ground truth match embedding for match t+1:
#             target_match_embd = self.match_embedding_module(
#                 team1_players[:, 1:, :, :],
#                 team2_players[:, 1:, :, :],
#                 match_info[:, 1:, :]
#             )  # (B, T-1, embedding_dim)

#             # Predict fantasy scores from the predicted next form combined with the target player and match embeddings.
#             # The fantasy score prediction module expects three tensors of shape (B, T-1, embedding_dim) and outputs (B, T-1, 1)
#             pred_fantasy = self.fantasy_score_prediction_module(pred_perf, target_player_embd, target_match_embd)
#             # Compute loss (e.g., MSE loss) between predicted fantasy scores and target.
#             loss = F.mse_loss(pred_fantasy.squeeze(-1), target)
#         return perf_emb, loss


In [ ]:
# experiment with the dropout layers
class FantasyScorePrediction(nn.Module):
    """Predict the Fantasy Score by the <CLS> embedding, the next match_embedding, and the player_embedding."""
    def __init__(self, embedding_dim=config.PERFORMANCE_EMBD_DIM):
        super().__init__()
        self.proj1 = nn.Linear(3 * config.PERFORMANCE_EMBD_DIM, 2048)
        self.proj2 = nn.Linear(2048, 512)
        self.proj3 = nn.Linear(512, 512)
        self.proj4 = nn.Linear(512, 256)
        self.proj5 = nn.Linear(256, 256)
        self.proj6 = nn.Linear(256, 128)
        self.proj7 = nn.Linear(128, 128)
        self.proj8 = nn.Linear(128, 1)

    def forward(self, x, target_player_embd, target_match_embd): 
        # Concatenate the embeddings along the last dimension
        # print(f"NI fantasy {x.shape} & {target_player_embd.shape} & {target_match_embd.shape}")
        x = torch.cat([x, target_player_embd, target_match_embd], dim=-1)

        x = F.gelu(self.proj1(x))
        x = F.gelu(self.proj2(x))
        x = F.gelu(self.proj3(x))
        x = F.gelu(self.proj4(x))
        x = F.gelu(self.proj5(x))
        x = F.gelu(self.proj6(x))
        x = F.gelu(self.proj7(x))


        x = self.proj8(x)

        return x


In [ ]:
"""Generalization: 
If your evaluation metric or deployment scenario is more 
sensitive to final outcomes rather than intermediate steps, training with a 
weighted loss that emphasizes these outcomes can enhance generalization.
"""
def custom_loss2(pred_fantasy,target,weights = None):
    """ 
        Auto regressive loss over T time steps.
        
        Inputs:
        - pred_fantasy: Predicted outputs of shape (B, T)
        - target: Ground truth values of shape (B, T)
        - weights : to each time step loss... (we can pre-compute them instead of computing them again -2)
        
        Output:
        - total_loss: A scalar tensor representing the weighted MSE loss
        
        We use a piecewise linear weighting scheme:
        - For t <= t0: weight = alpha
        - For t >  t0: weight = alpha + (1 - alpha) * (t - t0) / (T - t0)
        
        This formulation emphasizes later time steps (after a certain context is available)
        and can enhance generalization when final outcomes are more important.
    """
    B,T,_ = pred_fantasy.shape # shape (B,T,)
    alpha = 0.5
    t0 = 5  #TODO for time steps less then t0 , loss will have weights 0.5
    if weights is None:
        weights = []
        for t in range(1,T + 1):
            if t <=t0:
                weights.append(alpha)
            else:
                weight = alpha + (1 - alpha)*(t -t0)/(T - t0)
                weights.append(weight)
    weights = torch.tensor(weights,device=pred_fantasy.device,dtype=pred_fantasy.dtype)
    weights = weights.unsqueeze(0) # shape is (1,T)
    squared_error = (pred_fantasy - target)**2
    weighted_error = squared_error*weights
    loss = weighted_error.mean()
    return loss
        
        

class LogisticWeightedMSELoss(nn.Module):
    def __init__(self,T:int,W_MIN = 0.25,W_MAX = 1,k= 0.5):
        super(LogisticWeightedMSELoss,self).__init__()
        t = np.arange(1,T + 1)
        self.weights = W_MIN + (W_MAX - W_MIN)/ (1 + np.exp(-k * (t - T/2)))
        self.weights =  torch.tensor(weights, dtype=torch.float32)
        
    def forward(self,pred_fantasy,targets):
        """
        Args:
            pred_fantasy (torch.tensor): shape is (B,T)
            targets (torch.tensor): shape is (B,T)
        """
        squared_error = (pred_fantasy - targets)**2
        weighted_errors = squared_error*self.weights
        return weighted_errors
    
        
    
    
    

In [16]:
def custom_loss(pred_fantasy, target):
    # """
    # Custom Upper-Lower Bound Loss for integer fantasy scores.

    # For each example:
    #   - Define a lower bound = target - 0.5
    #   - Define an upper bound = target + 0.5
    #   - If pred_fantasy is within [lower, upper], loss = 0
    #   - If pred_fantasy < lower, loss = lower - pred_fantasy
    #   - If pred_fantasy > upper, loss = pred_fantasy - upper

    # Args:
    #     pred_fantasy (Tensor): Predicted fantasy scores, shape (B, ...), floats.
    #     target       (Tensor): Ground-truth fantasy scores, shape (B, ...), integers or floats.

    # Returns:
    #     Tensor: Scalar mean loss.
    # """
    # # convert target to float tensor
    # target = target.to(pred_fantasy.dtype)

    # # compute bounds
    # lower = target - 0.5
    # upper = target + 0.5

    # # zero tensor for comparison
    # zeros = torch.zeros_like(pred_fantasy)

    # # penalty for being below lower bound
    # loss_low = torch.where(pred_fantasy < lower,
    #                        lower - pred_fantasy,
    #                        zeros)

    # # penalty for being above upper bound
    # loss_high = torch.where(pred_fantasy > upper,
    #                         pred_fantasy - upper,
    #                         zeros)

    # # total loss per example
    # loss = loss_low + loss_high

    # return average
    # return loss.mean()
    
    # print(f"custom loss {pred_fantasy.shape} & {target.shape}")
    return nn.functional.mse_loss(pred_fantasy, target)


In [ ]:
player_embedding_module = PlayerEmbedding(
    in_channels=config.PLAYER_INPUT_DIM,
    out_channels=config.PERFORMANCE_EMBD_DIM
)

# Note: NextFormPredictor expects the *class* MatchEmbedding so that inside it
# it can do `match_embedding_module(player_embedding_module)`
match_embedding_module = MatchEmbedding

fantasy_score_prediction_module = FantasyScorePrediction(
    embedding_dim=config.PERFORMANCE_EMBD_DIM
)

model = NextFormPredictor(
    player_embedding_module=player_embedding_module,
    match_embedding_module=match_embedding_module,
    fantasy_score_prediction_module=fantasy_score_prediction_module,
    custom_loss=custom_loss,
    embedding_dim=config.PERFORMANCE_EMBD_DIM,
    num_layers=6,
    n_head=8
).to(config.DEVICE)



optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)


## Training

In [18]:
from tqdm import tqdm
from typing import List

def train(model, dataloader, start_epoch=0) -> List[torch.Tensor]:
    model.to(config.DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE)
    epoch_losses = []

    for epoch in range(start_epoch, config.NUM_EPOCHS):
        model.train()
        total_loss = 0.0

        # Wrap dataloader with tqdm for progress bar
        loop = tqdm(dataloader, leave=True)
        loop.set_description(f"Epoch [{epoch+1}/{config.NUM_EPOCHS}]")

        for batch in loop:
            # Move tensors to device
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch[k] = v.to(config.DEVICE)

            # Unpack batch
            player_univ   = batch['univ_features']
            context_feats = batch['context_matches']
            team1_players = batch['team1_players']
            team2_players = batch['team2_players']
            match_info    = batch['match_info']
            target_scores = batch['target_scores']
            
            # print(player_univ.shape)
            # print(context_feats.shape)
            # print(team1_players.shape)
            # print(team2_players.shape)
            # print(match_info.shape)
            # print(target_scores.shape)

            optimizer.zero_grad()
            perf_emb, loss = model(
                player_univ,
                context_feats,
                team1_players,
                team2_players,
                match_info,
                target_scores
            )
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update tqdm progress bar with current loss
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(dataloader)
        epoch_losses.append(avg_loss)

        print(f"Epoch {epoch+1}/{config.NUM_EPOCHS} — Avg Loss: {avg_loss:.4f}")

    return epoch_losses


In [ ]:
train(model, dataloader)

Epoch [1/100]: 100%|██████████| 19/19 [02:19<00:00,  7.35s/it, loss=2.88e+3]


Epoch 1/100 — Avg Loss: 3117.3702


Epoch [2/100]: 100%|██████████| 19/19 [01:54<00:00,  6.04s/it, loss=2.06e+3]


Epoch 2/100 — Avg Loss: 2156.3024


Epoch [3/100]: 100%|██████████| 19/19 [06:25<00:00, 20.27s/it, loss=1.74e+3]


Epoch 3/100 — Avg Loss: 1678.3442


Epoch [4/100]: 100%|██████████| 19/19 [04:50<00:00, 15.29s/it, loss=1.73e+3]


Epoch 4/100 — Avg Loss: 1636.9903


Epoch [5/100]:  32%|███▏      | 6/19 [01:33<03:07, 14.45s/it, loss=1.73e+3]

### IS CONTEXT_SIZE too big check how many players remain when we remove all the players having total matches less that of config.CONTEXT_LEN, check the input dim are as expected in class's forward function

In [ ]:
torch.save(model.state_dict(), "next_form_predictor.pth")
In match embd torch.Size([16, 16, 11, 25]) & torch.Size([16, 16, 14])

## Evalutaion

## Inference

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for a pretrained model (e.g., BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

text = "The patient was diagnosed with myocarditis and cardiomyopathy."
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

candidate_terms = []
i = 0
while i < len(tokens):
    token = tokens[i]
    # Check if the next token exists and starts with '##'
    if i < len(tokens) - 1 and tokens[i+1].startswith("##"):
        # Start forming the candidate term from the current token
        current_term = token
        # Advance index and merge subsequent subword tokens
        while i + 1 < len(tokens) and tokens[i+1].startswith("##"):
            i += 1
            current_term += tokens[i][2:]
        candidate_terms.append(current_term)
    i += 1

print("Candidate Terms:", candidate_terms)


In [ ]:
import re
from transformers import AutoTokenizer
from nltk import ngrams
from collections import Counter

# Step 1: Tokenize and merge subword tokens to extract candidate unknown words.
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = ("The patient was diagnosed with myocarditis and cardiomyopathy also pheeiong simenowon. "
        "The patient had chronic kidney disease and myocarditis symptoms along with the severe symptoms of the pheeiong simenowon.")
tokens = tokenizer.tokenize(text)
print("Tokens:", tokens)

# Extract candidate unknown words using subword merging.
candidate_terms = []
i = 0
while i < len(tokens):
    token = tokens[i]
    if i < len(tokens) - 1 and tokens[i+1].startswith("##"):
        current_term = token
        while i + 1 < len(tokens) and tokens[i+1].startswith("##"):
            i += 1
            current_term += tokens[i][2:]
        candidate_terms.append(current_term)
    i += 1

print("Candidate Terms from Subwords:", candidate_terms)

# Remove duplicates to get the unique set of new words.
new_words = list(set(candidate_terms))
print("Unique New Words:", new_words)

# Step 2: Re-examine the token stream to capture only those tokens (or merged tokens)
# that are in the new words set, preserving their order of occurrence.
unknown_sequence = []
i = 0
while i < len(tokens):
    token = tokens[i]
    # If token is the start of a merged word (has a following subword)
    if i < len(tokens) - 1 and tokens[i+1].startswith("##"):
        current_term = token
        while i + 1 < len(tokens) and tokens[i+1].startswith("##"):
            i += 1
            current_term += tokens[i][2:]
        # Append only if the merged token is one of the new words.
        if current_term in new_words:
            unknown_sequence.append(current_term)
    else:
        # For standalone tokens, check if they are in new words.
        if token in new_words:
            unknown_sequence.append(token)
    i += 1

print("Sequence of New Words in Order:", unknown_sequence)

# Step 3: Perform n-gram analysis on the sequence of new words.
# This will help detect if 2 or 3 new words are repeatedly appearing together.
# You can adjust the n-gram size and the frequency threshold as needed.
for n in [2, 3]:
    n_grams = list(ngrams(unknown_sequence, n))
    ngram_counts = Counter(n_grams)
    # For example, only consider phrases that appear at least twice.
    frequent_ngrams = [' '.join(gram) for gram, count in ngram_counts.items() if count >= 2]
    print(f"Frequent {n}-grams among new words:", frequent_ngrams)
